## Function Examples
Below are some C# implementations of various ways that function may appear in a code base

In [ ]:
// 

## Impure Function example

## Reimplemented as a pure function

## Side effects and how to deal with them

## Parallelizing pure functions is 🔥

### Honesty is the best policy
Writing a function that doesn't lie to me or others about what it does. 

### Voltron: creating a higher order function
e pluribus unum 

## Immutability in C# 10

## Immutability before C# 10

## Copy on update

### Bonus items

Removing `null` with `Option<T>`